In [1]:
"""
    TODO; Test passing to labels, r_type, token strings that have multiple items separated by commas
    It should be as easy as to send to push! instead of node.labels (edge.r_type, token.token) split(node.labels, ",")
"""

include("src//graph.jl")
include("src//tokens.jl")
include("src//search.jl")

using ..HllGraph
using ..HllSets
using ..Tokens
using ..Search

using Redis
using EasyConfig
using UUIDs
using Random

conn = Redis.RedisConnection()

HllGraph.node_idx(conn, "b:nodes", "b:n")
HllGraph.edge_idx(conn,"b:edges", "b:e")

ServerException("Index already exists")
ServerException("Index already exists")


In [2]:
# Initialize test HllSets
hll1 = HllSets.HllSet{10}()
hll2 = HllSets.HllSet{10}()
hll3 = HllSets.HllSet{10}()
hll4 = HllSets.HllSet{10}()
hll5 = HllSets.HllSet{10}()

# Generate datasets from random strings
s1 = Set(randstring(7) for _ in 1:10)
s2 = Set(randstring(7) for _ in 1:15)
s3 = Set(randstring(7) for _ in 1:100)
s4 = Set(randstring(7) for _ in 1:20)
s5 = Set(randstring(7) for _ in 1:130)

# Add datasets to HllSets
HllSets.add!(hll1, s1)
HllSets.add!(hll2, s2)
HllSets.add!(hll3, s3)
HllSets.add!(hll4, s4)
HllSets.add!(hll5, s5)

print(hll1)

HllSet{10}()


In [3]:
# Pr9nt cardinality of datasets and HllSets side by side
print(length(s1), " : ", HllSets.count(hll1), "\n")
print(length(s2), " : ", HllSets.count(hll2), "\n")
print(length(s3), " : ", HllSets.count(hll3), "\n")
print(length(s4), " : ", HllSets.count(hll4), "\n")
print(length(s5), " : ", HllSets.count(hll5), "\n\n")

# union
print(length(s1 ∪ s2 ∪ s3 ∪ s4 ∪ s5), " : ", HllSets.count(hll1 ∪ hll2 ∪ hll3 ∪ hll4 ∪ hll5), "\n")

# intersection
print(length(s1 ∩ s2 ∩ s3 ∩ s4 ∩ s5), " : ", HllSets.count(hll1 ∩ hll2 ∩ hll3 ∩ hll4 ∩ hll5), "\n")

10 : 9
15 : 13
100 : 103
20 : 21
130 : 125

275 : 272
0 : 1


In [4]:
node1 = HllGraph.Node(HllSets.id(hll1), "node1", HllSets.dump(hll1))
node2 = HllGraph.Node(HllSets.id(hll2), "node2", HllSets.dump(hll2))
node3 = HllGraph.Node(HllSets.id(hll3), "node3", HllSets.dump(hll3))
node4 = HllGraph.Node(HllSets.id(hll4), "node4", HllSets.dump(hll4))
node5 = HllGraph.Node(HllSets.id(hll5), "node5", HllSets.dump(hll5))

# println(node1.searchable)

graph = HllGraph.Graph([node1, node2, node3, node4, node5], [])

Main.HllGraph.Graph(Main.HllGraph.Node[Node(e7a8c5a4dd0058c7fad69d692e7a861aead4c1f6; node1; 9), Node(fb8ddcc9f7a437e40c1949d1654adecfee190791; node2; 13), Node(10c9b9c78de1ea14aa3dd6570766e30fd4e954b5; node3; 103), Node(c811b3ff88b5f00b64a4ff7f221b87e0a63a83b0; node4; 21), Node(6df766163561ef69810335f1469e83f8b3561f99; node5; 125)], Main.HllGraph.Edge[])

In [5]:
using JSON3

HllGraph.set_node(conn, node1, "b")
HllGraph.set_node(conn, node2, "b")
HllGraph.set_node(conn, node3, "b")
HllGraph.set_node(conn, node4, "b")
HllGraph.set_node(conn, node5, "b")
# println(JSON3.read(s1, Set))
Tokens.set_tokens(conn, s1, node1.sha1, "b")
Tokens.set_tokens(conn, s2, node2.sha1, "b")
Tokens.set_tokens(conn, s3, node3.sha1, "b")
Tokens.set_tokens(conn, s4, node4.sha1, "b")
Tokens.set_tokens(conn, s5, node5.sha1, "b")

In [6]:
println(HllGraph.get_node(conn, node1.sha1, "b"))
println(node1.sha1)
println(node1.labels)

nodes = HllGraph.get_nodes(conn, "b")
edges = HllGraph.get_edges(conn, "b")
println(nodes)

println(edges)

HllGraph.Graph(nodes, edges)

Node(e7a8c5a4dd0058c7fad69d692e7a861aead4c1f6; node1; 9)
e7a8c5a4dd0058c7fad69d692e7a861aead4c1f6
node1
Any[Node(d7141bbc7890690f41879dfb696bdb0141b882e8; node2; 13), Node(6df766163561ef69810335f1469e83f8b3561f99; node5; 125), Node(59308068f6f7d3ef1343d5914916d78f091ae2cd; union; 23), Node(e7a8c5a4dd0058c7fad69d692e7a861aead4c1f6; node1; 9), Node(fb8ddcc9f7a437e40c1949d1654adecfee190791; node2; 13), Node(05673e6ae438274cb50cf51f22b87c57511ebf4f; node4; 21), Node(c811b3ff88b5f00b64a4ff7f221b87e0a63a83b0; node4; 21), Node(6c3a38242fab0947c2783d0df4317780d1b36ff4; node1; 10), Node(6286bfd326d0348969d930791d8463e5367d7022; node5; 133), Node(39e6fc374b0caec6540e2a16e9bdda80be4e9031; node3; 102), Node(10c9b9c78de1ea14aa3dd6570766e30fd4e954b5; node3; 103)]
Any[Edge(6c3a38242fab0947c2783d0df4317780d1b36ff4, 59308068f6f7d3ef1343d5914916d78f091ae2cd, "union"), Edge(d7141bbc7890690f41879dfb696bdb0141b882e8, 59308068f6f7d3ef1343d5914916d78f091ae2cd, "union")]


Main.HllGraph.Graph(Main.HllGraph.Node[Node(d7141bbc7890690f41879dfb696bdb0141b882e8; node2; 13), Node(6df766163561ef69810335f1469e83f8b3561f99; node5; 125), Node(59308068f6f7d3ef1343d5914916d78f091ae2cd; union; 23), Node(e7a8c5a4dd0058c7fad69d692e7a861aead4c1f6; node1; 9), Node(fb8ddcc9f7a437e40c1949d1654adecfee190791; node2; 13), Node(05673e6ae438274cb50cf51f22b87c57511ebf4f; node4; 21), Node(c811b3ff88b5f00b64a4ff7f221b87e0a63a83b0; node4; 21), Node(6c3a38242fab0947c2783d0df4317780d1b36ff4; node1; 10), Node(6286bfd326d0348969d930791d8463e5367d7022; node5; 133), Node(39e6fc374b0caec6540e2a16e9bdda80be4e9031; node3; 102), Node(10c9b9c78de1ea14aa3dd6570766e30fd4e954b5; node3; 103)], Main.HllGraph.Edge[Edge(6c3a38242fab0947c2783d0df4317780d1b36ff4, 59308068f6f7d3ef1343d5914916d78f091ae2cd, "union"), Edge(d7141bbc7890690f41879dfb696bdb0141b882e8, 59308068f6f7d3ef1343d5914916d78f091ae2cd, "union")])

In [7]:
union_node = HllGraph.union_nodes(conn, node1, node2, "union", "b")

Node(e3fae71efa879a113f6eee07e71aec8b5dda451a; union; 22)

In [8]:
println(HllGraph.get_edges(conn, "b"))

Any[Edge(6c3a38242fab0947c2783d0df4317780d1b36ff4, 59308068f6f7d3ef1343d5914916d78f091ae2cd, "union"), Edge(e7a8c5a4dd0058c7fad69d692e7a861aead4c1f6, e3fae71efa879a113f6eee07e71aec8b5dda451a, "union"), Edge(fb8ddcc9f7a437e40c1949d1654adecfee190791, e3fae71efa879a113f6eee07e71aec8b5dda451a, "union"), Edge(d7141bbc7890690f41879dfb696bdb0141b882e8, 59308068f6f7d3ef1343d5914916d78f091ae2cd, "union")]


In [9]:
value ="0000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000"

println(length(value))

1024
